In [89]:
import pandas as pd
%run heartbeat_slicer.ipynb

In [90]:
from gtda.time_series import SingleTakensEmbedding
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_point_cloud
from datetime import datetime

In [91]:
folder_path = "data_processed/svdb"
record_names_path = "data_original/svdb/RECORDS"

with open(record_names_path, 'r') as record_names_file:
    record_names = record_names_file.readlines()

record_names = [record_name[:-1] for record_name in record_names]

In [92]:
dataSvDB = []

for record_name in record_names:
    sliced = HeartbeatSlicer(folder_path + '/' + record_name + '_processed')
    dataSvDB.append(sliced.annotate_heartbeats())

In [96]:
#[patient][heartbeat_num] - [0]:heartbeat [1]:ann
dataSvDB[2][3][0].shape

AttributeError: 'list' object has no attribute 'shape'

In [97]:
def make_point_cloud(heartbeat_):

    embedding_dimension_periodic = 3
    embedding_time_delay_periodic = 8
    embedder_periodic = SingleTakensEmbedding(
        parameters_type="fixed",
        n_jobs=2,
        time_delay=embedding_time_delay_periodic,
        dimension=embedding_dimension_periodic,
    )
    point_cloud_ = embedder_periodic.fit_transform(heartbeat)
    plot_point_cloud(point_cloud_)

    return point_cloud_


In [98]:
def make_complex_w_pers_diag(point_cloud_):
    point_cloud_ = point_cloud_[None, :, :]

    homology_dimensions = [0, 1]

    periodic_persistence = VietorisRipsPersistence(
        homology_dimensions=homology_dimensions, n_jobs=-1
    )

    diag_ = periodic_persistence.fit_transform(point_cloud_)

    return diag_


In [99]:
def make_betti_curve(diag_):
    bc = BettiCurve(n_jobs=-1)
    betti_curve_ = bc.fit_transform(diag_)
    return betti_curve_

In [89]:
# betti = make_betti_curve(dataMitDB[2][2][0])

In [90]:
# betti[0][0]

In [ ]:
df_to_be = []
patient_counter = 0
patient_number = len(dataSvDB)
timestamp_begin = datetime.now()

for patient in dataSvDB:
    patient_counter = patient_counter+1
    heartbeat_counter = 0
    heartbeat_number = len(patient)

    for heartbeat, ann in patient:
        heartbeat_counter = heartbeat_counter+1
        print("Processing patient: ", patient_counter, "/", patient_number, " | heartbeat: ", heartbeat_counter, "/", heartbeat_number)
        # Convert the segment to a point cloud
        point_cloud = make_point_cloud(heartbeat)

        # Compute the Vietoris-Rips complex and its persistence diagram
        diag = make_complex_w_pers_diag(point_cloud)

        # Compute and return Betti curve
        betti = make_betti_curve(diag)

        df_to_be.append([point_cloud, diag, betti, ann])


timestamp_end = datetime.now()
print("----------------------------------------------------------------")
print("started: ", timestamp_begin, " | finished: ", timestamp_end)
df = pd.DataFrame(df_to_be, columns=['points_cloud', 'diag', 'betti', 'ann'])


Processing patient:  1 / 78  | heartbeat:  1 / 1915
Processing patient:  1 / 78  | heartbeat:  2 / 1915
Processing patient:  1 / 78  | heartbeat:  3 / 1915
Processing patient:  1 / 78  | heartbeat:  4 / 1915
Processing patient:  1 / 78  | heartbeat:  5 / 1915
Processing patient:  1 / 78  | heartbeat:  6 / 1915
Processing patient:  1 / 78  | heartbeat:  7 / 1915
Processing patient:  1 / 78  | heartbeat:  8 / 1915
Processing patient:  1 / 78  | heartbeat:  9 / 1915
Processing patient:  1 / 78  | heartbeat:  10 / 1915
Processing patient:  1 / 78  | heartbeat:  11 / 1915
Processing patient:  1 / 78  | heartbeat:  12 / 1915
Processing patient:  1 / 78  | heartbeat:  13 / 1915
Processing patient:  1 / 78  | heartbeat:  14 / 1915
Processing patient:  1 / 78  | heartbeat:  15 / 1915
Processing patient:  1 / 78  | heartbeat:  16 / 1915
Processing patient:  1 / 78  | heartbeat:  17 / 1915
Processing patient:  1 / 78  | heartbeat:  18 / 1915
Processing patient:  1 / 78  | heartbeat:  19 / 1915
Pr

In [53]:
df.to_csv("data_processed/svDB.csv")